In [ ]:
import os
import sys
import pandas as pd
from pathlib import Path
import logging
import traceback

# --- Configuration du Logging ---
logger = logging.getLogger("notebook_feature_engineering")
if not logger.handlers:
    handler = logging.StreamHandler(sys.stdout) # Afficher les logs dans la sortie du notebook
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)

# --- Configuration des Chemins (depuis la racine du projet) ---
# On suppose que ce notebook est à la racine du projet: /home/morningstar/Desktop/crypto_robot/Morningstar/
try:
    PROJECT_ROOT = Path(os.getcwd()).resolve()
    if PROJECT_ROOT.name != "Morningstar": # Petite vérification
        logger.warning(f"Le répertoire actuel ({PROJECT_ROOT}) ne semble pas être la racine 'Morningstar'. Ajustez PROJECT_ROOT si nécessaire.")
        # Si tu es sûr d'être à la racine, tu peux forcer :
        # PROJECT_ROOT = Path("/home/morningstar/Desktop/crypto_robot/Morningstar").resolve()
except Exception as e:
    logger.error(f"Erreur lors de la détermination de PROJECT_ROOT: {e}")
    # Fallback si os.getcwd() pose problème dans certains environnements de notebook
    PROJECT_ROOT = Path("/home/morningstar/Desktop/crypto_robot/Morningstar").resolve()

logger.info(f"PROJECT_ROOT détecté/défini à : {PROJECT_ROOT}")

# Ajouter la racine du projet au sys.path pour permettre les imports de 'ultimate'
# Cela permet d'importer des modules comme `from ultimate.utils...`
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
    logger.info(f"Ajout de {PROJECT_ROOT} à sys.path pour les imports.")

# Importer la fonction de feature engineering
try:
    from ultimate.utils.feature_engineering import apply_feature_pipeline
    logger.info("Import de 'apply_feature_pipeline' réussi.")
except ImportError as e:
    logger.error(f"Impossible d'importer 'apply_feature_pipeline': {e}")
    logger.error("Assurez-vous que PROJECT_ROOT est correct et que la structure de votre projet permet cet import.")
    apply_feature_pipeline = None # Pour éviter des erreurs plus loin si l'import échoue

# Définir les répertoires d'entrée et de sortie
# CORRECTION ICI :
RAW_CSV_DATA_DIR = PROJECT_ROOT / "ultimate" / "data" # Les CSV sont directement dans ultimate/data/
PROCESSED_FEATURES_DIR = PROJECT_ROOT / "ultimate" / "data" / "processed" / "market_features"

# Créer le répertoire de destination s'il n'existe pas
PROCESSED_FEATURES_DIR.mkdir(parents=True, exist_ok=True)
logger.info(f"Répertoire des données brutes CSV : {RAW_CSV_DATA_DIR}")
logger.info(f"Répertoire de sauvegarde des features Parquet : {PROCESSED_FEATURES_DIR}")

# Liste des fichiers CSV à traiter (adapte cette liste si nécessaire)
# On se base sur les fichiers que tu as montrés précédemment.
csv_files_to_process = [
    "ADAUSDT_1m.csv",
    "BTCUSDT_1m.csv",
    "BNBUSDT_1m.csv",
    "ETHUSDT_1m.csv",
    "XRPUSDT_1m.csv"
]

if apply_feature_pipeline: # Ne continuer que si l'import a réussi
    for csv_filename in csv_files_to_process:
        csv_path = RAW_CSV_DATA_DIR / csv_filename
        
        if not csv_path.exists():
            logger.warning(f"Le fichier {csv_path} n'a pas été trouvé. Ignoré.")
            continue
            
        try:
            logger.info(f"\n--- Traitement du fichier : {csv_filename} ---")
            
            # Extrait le symbole du nom du fichier (ex: 'BTCUSDT' de 'BTCUSDT_1m.csv')
            symbol = csv_filename.replace("_1m.csv", "").replace(".csv", "")
            logger.info(f"  Symbole extrait : {symbol}")

            # 1. Lire le fichier CSV brut
            df_raw = pd.read_csv(csv_path)
            logger.info(f"  Données brutes chargées. Shape: {df_raw.shape}")

            if df_raw.empty:
                logger.warning(f"  ATTENTION: Le fichier {csv_filename} est vide. Ignoré.")
                continue

            # Assurer la présence des colonnes requises
            required_columns = {"timestamp", "open", "high", "low", "close", "volume"}
            if not required_columns.issubset(df_raw.columns):
                logger.error(f"  ERREUR: Colonnes manquantes dans {csv_filename}. Requis: {required_columns}. Colonnes trouvées: {df_raw.columns.tolist()}")
                # Tu peux ajouter ici une logique de renommage si tes colonnes CSV ont des noms différents
                # Exemple: df_raw.rename(columns={'Date': 'timestamp', 'Open': 'open', ...}, inplace=True)
                continue 
            
            # Convertir 'timestamp' en datetime
            try:
                df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])
            except Exception as e_time:
                logger.error(f"  ERREUR: Impossible de convertir la colonne 'timestamp' en datetime pour {csv_filename}: {e_time}. Ignoré.")
                continue
            
            # Ajouter la colonne 'symbol' si elle n'existe pas (utile pour MultiAssetEnv et le feature engineering)
            if 'symbol' not in df_raw.columns:
                df_raw['symbol'] = symbol
                logger.info(f"  Colonne 'symbol' ajoutée avec la valeur '{symbol}'.")

            # 2. Appliquer le pipeline de feature engineering
            # La fonction apply_feature_pipeline devrait s'occuper des indicateurs techniques et HMM.
            # Elle met 'timestamp' en index.
            logger.info(f"  Application du pipeline de feature engineering...")
            df_features = apply_feature_pipeline(df_raw.copy(), include_llm=False) # include_llm=False pour l'entraînement RL typique

            if df_features is None or df_features.empty:
                logger.warning(f"  ATTENTION: Le pipeline de feature engineering n'a rien retourné pour {csv_filename}. Ignoré.")
                continue
            
            logger.info(f"  Features techniques et HMM appliquées. Shape après pipeline: {df_features.shape}")

            # 3. S'assurer que 'timestamp' est une colonne pour la sauvegarde
            if isinstance(df_features.index, pd.DatetimeIndex) and 'timestamp' not in df_features.columns:
                df_features = df_features.reset_index()
            
            if 'timestamp' not in df_features.columns:
                logger.error(f"  ERREUR: Colonne 'timestamp' manquante après le pipeline pour {csv_filename}. Ignoré.")
                continue
            
            # S'assurer que la colonne timestamp est bien de type datetime64[ns]
            df_features['timestamp'] = pd.to_datetime(df_features['timestamp'])

            # 4. Définir le chemin de sortie et sauvegarder au format Parquet
            output_filename = f"{symbol}_features.parquet" # Nom basé sur le symbole
            output_path = PROCESSED_FEATURES_DIR / output_filename
            
            df_features.to_parquet(output_path, index=False)
            logger.info(f"  ✅ Fichier traité sauvegardé : {output_path}")
            logger.info(f"     Shape finale : {df_features.shape}")

        except Exception as e:
            logger.error(f"  ERREUR inattendue lors du traitement de {csv_filename}: {e}")
            logger.error(f"  Traceback complet:\n{traceback.format_exc()}")

    logger.info("\nTraitement de tous les fichiers CSV terminé.")
else:
    logger.error("Le script ne peut pas continuer car 'apply_feature_pipeline' n'a pas pu être importé.")


2025-05-23 15:49:31,498 - notebook_feature_engineering - INFO - PROJECT_ROOT détecté/défini à : /home/morningstar/Desktop/crypto_robot/Morningstar
2025-05-23 15:49:31,499 - notebook_feature_engineering - INFO - Ajout de /home/morningstar/Desktop/crypto_robot/Morningstar à sys.path pour les imports.
2025-05-23 15:49:31,499 - notebook_feature_engineering - INFO - Import de 'apply_feature_pipeline' réussi.
2025-05-23 15:49:31,500 - notebook_feature_engineering - INFO - Répertoire des données brutes CSV : /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/raw
2025-05-23 15:49:31,500 - notebook_feature_engineering - INFO - Répertoire de sauvegarde des features Parquet : /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/processed/market_features
2025-05-23 15:49:31,501 - notebook_feature_engineering - WARNING - Le fichier /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/raw/ADAUSDT_1m.csv n'a pas été trouvé. Ignoré.
2025-05-23 15:49:31,501 - noteb

In [ ]:
import pandas as pd
from pathlib import Path
from ultimate.utils.feature_engineering import apply_feature_pipeline
from ultimate.data_processors.hmm_regime_detector import HMMRegimeDetector

RAW_DIR = Path("/home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/raw/market")
FEATURE_DIR = RAW_DIR / "features" / "feature"
FEATURE_DIR.mkdir(parents=True, exist_ok=True)

for parquet_path in RAW_DIR.glob("*.parquet"):
    print(f"Étape 1 - Traitement de {parquet_path.name}")
    df = pd.read_parquet(parquet_path)
    df = apply_feature_pipeline(df, include_llm=False)

    # Ajout HMM (régimes de marché)
    hmm = HMMRegimeDetector()
    df['return'] = df['close'].pct_change().fillna(0)
    regimes, probs = hmm.detect_regimes(df['return'].values)
    df['hmm_regime'] = regimes
    for i in range(probs.shape[1]):
        df[f"hmm_prob_{i}"] = probs[:, i]
    df.drop(columns=['return'], inplace=True)

    # Assure la présence de 'timestamp' en colonne
    if isinstance(df.index, pd.DatetimeIndex) or 'timestamp' not in df.columns:
        df = df.reset_index()
    if 'timestamp' not in df.columns:
        raise RuntimeError(f"'timestamp' column missing after feature pipeline for {parquet_path.name}")

    out_path = FEATURE_DIR / f"{parquet_path.stem}_features.parquet"
    df.to_parquet(out_path, index=False)
    print(f"✅ {out_path.name} shape: {df.shape}")

In [ ]:
import pandas as pd
from pathlib import Path
from ultimate.utils.feature_engineering import apply_feature_pipeline
from ultimate.data_processors.hmm_regime_detector import HMMRegimeDetector

RAW_DIR = Path("/home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/raw/market")
FEATURE_DIR = RAW_DIR / "features" / "feature"
FEATURE_DIR.mkdir(parents=True, exist_ok=True)

for parquet_path in RAW_DIR.glob("*.parquet"):
    print(f"Étape 1 - Traitement de {parquet_path.name}")
    df = pd.read_parquet(parquet_path)
    df = apply_feature_pipeline(df, include_llm=False)

    # Ajout HMM (régimes de marché)
    hmm = HMMRegimeDetector()
    df['return'] = df['close'].pct_change().fillna(0)
    regimes, probs = hmm.detect_regimes(df['return'].values)
    df['hmm_regime'] = regimes
    for i in range(probs.shape[1]):
        df[f"hmm_prob_{i}"] = probs[:, i]
    df.drop(columns=['return'], inplace=True)

    # Assure la présence de 'timestamp' en colonne
    if isinstance(df.index, pd.DatetimeIndex) or 'timestamp' not in df.columns:
        df = df.reset_index()
    if 'timestamp' not in df.columns:
        raise RuntimeError(f"'timestamp' column missing after feature pipeline for {parquet_path.name}")

    out_path = FEATURE_DIR / f"{parquet_path.stem}_features.parquet"
    df.to_parquet(out_path, index=False)
    print(f"✅ {out_path.name} shape: {df.shape}")

In [ ]:
import pandas as pd
from pathlib import Path
from ultimate.utils.feature_engineering import apply_feature_pipeline
from ultimate.data_processors.hmm_regime_detector import HMMRegimeDetector

# Répertoire contenant les fichiers d'origine (.parquet)
RAW_DIR = Path("/home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/raw/market")

# Boucle sur chaque fichier .parquet dans le dossier
for parquet_path in RAW_DIR.glob("*.parquet"):
    print(f"🔄 Étape 1 - Traitement du fichier : {parquet_path.name}")

    # Lecture du fichier
    try:
        df = pd.read_parquet(parquet_path)
    except Exception as e:
        print(f"❌ Erreur de lecture {parquet_path.name} : {e}")
        continue

    # Application des features classiques (sans LLM ni MCP)
    try:
        df = apply_feature_pipeline(df, include_llm=False)
    except Exception as e:
        print(f"❌ Erreur pendant le pipeline de features pour {parquet_path.name} : {e}")
        continue

    # Ajout de la détection de régime HMM
    try:
        hmm = HMMRegimeDetector()
        df['return'] = df['close'].pct_change().fillna(0)
        regimes, probs = hmm.detect_regimes(df['return'].values)
        df['hmm_regime'] = regimes
        for i in range(probs.shape[1]):
            df[f"hmm_prob_{i}"] = probs[:, i]
        df.drop(columns=['return'], inplace=True)
    except Exception as e:
        print(f"⚠️ Erreur HMM pour {parquet_path.name} : {e}")
        continue

    # Vérifie et ajuste la colonne timestamp
    if isinstance(df.index, pd.DatetimeIndex):
        df = df.reset_index()
    if 'timestamp' not in df.columns:
        print(f"❌ Colonne 'timestamp' manquante dans {parquet_path.name}")
        continue

    # Construction du chemin de sortie
    out_path = RAW_DIR / f"{parquet_path.stem}_features.parquet"
    
    # Sauvegarde du fichier avec les features
    try:
        df.to_parquet(out_path, index=False)
        print(f"✅ Sauvegardé : {out_path.name} | shape = {df.shape}")
    except Exception as e:
        print(f"❌ Erreur de sauvegarde {out_path.name} : {e}")


In [ ]:
from ultimate.data_processors.cryptobert_processor import CryptoBERTProcessor

EMB_DIR = RAW_DIR / "features" / "emb"
EMB_DIR.mkdir(parents=True, exist_ok=True)
cryptobert = CryptoBERTProcessor()

for parquet_path in FEATURE_DIR.glob("*_features.parquet"):
    print(f"Étape 2 - Embedding pour {parquet_path.name}")
    df = pd.read_parquet(parquet_path)
    # Assure la présence de 'timestamp' en colonne
    if isinstance(df.index, pd.DatetimeIndex) or 'timestamp' not in df.columns:
        df = df.reset_index()
    if 'timestamp' not in df.columns:
        raise RuntimeError(f"'timestamp' column missing in {parquet_path.name}")

    # Génère un texte par ligne si besoin
    if "cryptotext" not in df.columns:
        df['cryptotext'] = df.apply(lambda row: f"{row['pair']} {row['timestamp']} close:{row['close']}", axis=1)
    texts = df['cryptotext'].astype(str).tolist()
    embeddings = cryptobert.generate_embeddings(texts)
    for i in range(10):  # Suppose 10 dims
        df[f"bert_{i}"] = [emb[i] if len(emb) > i else None for emb in embeddings]
    df["llm_embedding"] = df[[f"bert_{i}" for i in range(10)]].apply(lambda row: list(row), axis=1)
    out_path = EMB_DIR / f"{parquet_path.stem}_emb.parquet"
    df.to_parquet(out_path, index=False)
    print(f"✅ {out_path.name} shape: {df.shape}")

In [ ]:
FINAL_PATH = RAW_DIR / "features" / "dataset_enriched.parquet"
dfs = []
for parquet_path in EMB_DIR.glob("*_emb.parquet"):
    df = pd.read_parquet(parquet_path)
    # Vérification de la présence de colonnes-clés
    if 'timestamp' not in df.columns or 'pair' not in df.columns:
        raise RuntimeError(f"Missing key columns in {parquet_path.name}")
    dfs.append(df)
    print(f"Fusion - {parquet_path.name} shape: {df.shape}")

df_final = pd.concat(dfs, ignore_index=True)
df_final = df_final.sort_values(['timestamp', 'pair']).reset_index(drop=True)
df_final.to_parquet(FINAL_PATH, index=False)
print(f"✅ Dataset final fusionné: {FINAL_PATH} shape: {df_final.shape}")

NameError: name 'EMB_DIR' is not defined